In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import csv
import re
import random
import tqdm.notebook as tq
import pandas as pd
import json
import pprint

Mounted at /content/drive


## 1. 讀入語料與構式辭典

In [ ]:
file_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/threads/cht-2020-merged.txt'

with open(file_path, 'r', encoding = 'utf-8') as f:
  comments = [ line.strip() for line in f ] # comments 中的每一個 element 是一則評論

In [ ]:
construction_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list/data/constructions_0406.csv'

with open(construction_path) as f:
    constructions = [ row.strip() for row in f ]

## 2. 從每則評論中找出符合的構式例子

In [ ]:
n = 5  # sample size
random.seed(10)

matched = {}
matched_con = {}
for k in tq.tqdm(constructions):
  pat = re.compile(k)
  if k not in matched: matched[k] = []
  
  # Get all cnstr from all comments
  candidates = set()
  for cmt in comments:
    for c in pat.finditer(cmt): candidates.add(c[0])
  
  # Count all cnstr
  matched_con[k] = len(candidates)

  # Sample
  if len(candidates) > n:
    for c in random.sample(candidates, n): matched[k].append(c)
  else:
    for c in candidates: matched[k].append(c)

In [ ]:
constructions[1]

'純噓\\w+\\b'

In [ ]:
pprint.pprint(matched)

{'(對\\w+)?一點(都|也)不\\w+\\b': ['一點都不貴',
                             '一點都不考慮',
                             '一點都不深奧深奧的地方在價錢',
                             '一點都不意外',
                             '一點都不值得'],
 '(很|也)夠\\w+了\\b': ['也夠買一隻平價機了', '也夠優了', '也夠鄉下了', '很夠了應該也不會升就是了', '也夠打94分鐘了'],
 '(真的)?有夠\\w+的?\\b': ['有夠正',
                      '有夠猛的',
                      '有夠沒誠意',
                      '有夠差一場正常ping一場爆ping要怎麼玩',
                      '真的有夠低能'],
 '(置|拉)板凳等\\w+\\b': ['置板凳等後續', '拉板凳等中華'],
 '\\b\\w+(到|的|得)離譜': ['上傳更是低得離譜',
                      '寫的資料錯的離譜',
                      '開台就是貴的離譜',
                      '邏輯到離譜',
                      '小弟住在一中商圈方案為488吃到飽之前都是正常的但最近幾天網速慢到離譜'],
 '\\b\\w+(掰|bye)了': ['我想就掰了',
                     '出了市區偏遠地區如海邊山上就掰掰了',
                     '就要跟你說掰掰了',
                     '所以要和便宜的吃到飽說掰了',
                     '門號跟你說byebye了'],
 '\\b\\w+(的|得)要死': ['上網恨慢得要死', '幹的要死', '我是在二樓網路真的爛的要死', '網速慢的要死', '態度還差的要死'],
 '\\b\\w+到\\w*炸了?': ['幹來台南生活到哪都容易沒訊號稍微進一個建築物網路就直接炸'

In [ ]:
# export as csv file
with open('matched_random.csv', 'w') as f:
    for key in matched.keys():
        f.write("%s,%s\n"%(key, matched[key]))

## 3. 將符合構式的例子及其數量整理成表格

In [ ]:
df = pd.DataFrame([matched_con, matched]).T
df.columns = ['count', 'examples']
df = df.sort_values(by='count', ascending=False) # 數量由大到小排序
df

,count,examples
直接\w+了\b,672,"[直接裝網路好了, 直接掛斷了, 直接5g就不會有煩腦了, 直接變成空號了, 直接對半砍了]"
(真的)?有夠\w+的?\b,464,"[有夠正, 有夠猛的, 有夠沒誠意, 有夠差一場正常ping一場爆ping要怎麼玩, 真的有..."
也太\w+了?吧?\b,408,"[也太大吧, 也太划算了, 也太慘了, 也太多種, 也太多白痴了]"
\b\w+根本\w{2}\b,292,"[店家根本違約, 前面的人網購操作很慢根本悲劇, wifi路由器wan燈根本沒亮, 用不完的..."
坐等\w+\b,188,"[坐等降價囉, 坐等看後續有無10, 坐等今年雙11各家方案, 坐等明雙11, 坐等單辦88]"
\b\w+到\w*炸了?,162,"[幹來台南生活到哪都容易沒訊號稍微進一個建築物網路就直接炸, 真的是爛到一個炸, 慢到炸, ..."
(對\w+)?一點(都|也)不\w+\b,133,"[一點都不貴, 一點都不考慮, 一點都不深奧深奧的地方在價錢, 一點都不意外, 一點都不值得]"
\b\w+屌打\w+\b,113,"[那我想被原本戲謔但第二屆認真搞的走鐘獎屌打也是應該的看到其中一則留言寫得很好, 中華屌打台..."
\b\w+(的|得)要死,105,"[上網恨慢得要死, 幹的要死, 我是在二樓網路真的爛的要死, 網速慢的要死, 態度還差的要死]"
\b\w+給力\b,87,"[呱張給力, 我看前面兩篇都說399給力, 88結最給力, 又覺得中華網路還是給力, iph..."


## 4. 用非電信語料 (PTT 男女板) 測試構式辭典

In [ ]:
!gdown --id "1Aoeq6dPkgHhxJQonqPtyVPwPoRhpesPC" -O "bg.json" 
with open("bg.json", "r", encoding = "UTF-8") as f:
    bg_articles = json.load(f)['articles']

bg_content = []
for a in bg_articles:
  if 'content' in a:
    bg_content.append(a['content'])

Downloading...
From: https://drive.google.com/uc?id=1Aoeq6dPkgHhxJQonqPtyVPwPoRhpesPC
To: /content/bg.json
27.3MB [00:00, 104MB/s] 


In [ ]:
n = 5  # sample size
random.seed(10)

bg_matched = {}
bg_matched_con = {}
for k in tq.tqdm(constructions):
  pat = re.compile(k)
  if k not in bg_matched: bg_matched[k] = []
  
  # Get all cnstr from all comments
  candidates = set()
  for cmt in bg_content:
    for c in pat.finditer(cmt): candidates.add(c[0])
  
  # Count all cnstr
  bg_matched_con[k] = len(candidates)

  # Sample
  if len(candidates) > n:
    for c in random.sample(candidates, n): bg_matched[k].append(c)
  else:
    for c in candidates: bg_matched[k].append(c)

In [ ]:
bg_df = pd.DataFrame([bg_matched_con, bg_matched]).T
bg_df.columns = ['count', 'examples']
bg_df = bg_df.sort_values(by='count', ascending=False) # 數量由大到小排序
bg_df

,count,examples
直接\w+了\b,33,"[直接被已讀了, 直接不演了, 直接就不見了, 直接把軟體刪了, 直接ff找下個對象就好了]"
(對\w+)?一點(都|也)不\w+\b,30,"[一點也不了解你的本質, 一點也不快樂, 一點都不是, 一點都不現實, 一點都不期待不興奮也..."
(真的)?有夠\w+的?\b,30,"[有夠沒種的, 真的有夠白癡, 有夠衰, 有夠可怕, 有夠廢的你為何不離開]"
也太\w+了?吧?\b,23,"[也太神, 也太自然了吧, 也太巧合了吧, 也太多, 也太沒誠意]"
誰\w誰\w+\b,16,"[誰誰誰對老婆女友超級好超Beta人家感情也超好的啊, 誰對誰錯還真的不好說, 誰對誰錯只從..."
\b\w+根本\w{2}\b,16,"[覺得根本被玩, 你根本沒轍, 如果根本不熟, 要說誰是為了性別平等而努力根本沒有, 現在想..."
八成\w+\b,15,"[八成以上又是對方先找我聊天的, 八成左右, 八成婚姻就已經有問題了, 八成確定這是真的理由..."
\b\w+(的|得)要死,14,"[在台灣罵得要死, 畢竟社會上還是很多人幼稚得要死, 摳門小氣的要死, 還我一個禮拜我難過的..."
越來越\w\b,13,"[越來越忙, 越來越少, 越來越低, 越來越廢, 越來越近]"
\b\w+到\w*炸了?,10,"[浪漫到炸, 煩到爆炸, 錢多到爆炸, 現在想想如果交往第14天跟她求婚他應該會傻眼到爆炸,..."
